# Baseball learning network

In [1]:
import numpy as np
import tensorflow as tf
import tflearn
from tqdm import tqdm_notebook
import copy
from scipy import stats

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [1]:
class GameStats(object):          
    
    def __init__(self, homeTeamNameIndex, homeTeamScoreIndex, homeTeamStatsIndex, visitorTeamNameIndex, visitorTeamScoreIndex, visitorTeamStatsIndex):
        #parse the text file
        self.statsFile = open("baseball2016.txt", "r")
        self.topArray = []
        self.sideArray = []  
        self.sc = np.zeros((30,30,30), np.int32) 
        self.sc[:,:,:] = -1  
        self.am = np.zeros((30,30), np.float32)
        self.gameList = []
        
        seenTeams = []
        for line in self.statsFile:
            token = line.split(',')  #tokenize the string
            tokenIndex = [homeTeamNameIndex, homeTeamScoreIndex, visitorTeamNameIndex, visitorTeamScoreIndex] + [i for i in homeTeamStatsIndex] + [i for i in visitorTeamStatsIndex]
            attributes = dict()
            
            for i in range(len(token)):
                if(i in tokenIndex):
                    attributes[i] = self.removeQuotes(token[i])
                        
            self.addScore(attributes[homeTeamNameIndex], attributes[visitorTeamNameIndex], attributes[homeTeamScoreIndex], attributes[visitorTeamScoreIndex])                
            self.addGame(attributes[homeTeamNameIndex], attributes[homeTeamScoreIndex], [attributes[i] for i in homeTeamStatsIndex], attributes[visitorTeamNameIndex], attributes[visitorTeamScoreIndex], [attributes[i] for i in visitorTeamStatsIndex])
            
            if(attributes[homeTeamNameIndex] not in seenTeams):
                seenTeams.append(attributes[homeTeamNameIndex])
            
        self.buildAvgMatrix()
        self.statsFile.close()
        #self.gameList = [bin(2**i)[2:].zfill(len(seenTeams)) if x == seenTeams[i] else x for i in range(len(seenTeams)) for x in self.gameList]
        # take the teams and convert the indexes of the order that they appeared in into the hot one format by rasiing
        # 2 to the power of them and then converting them into binary.
        seenTeamsDict = {k: v for v, k in enumerate(seenTeams)}
        temp = []
        for x in self.gameList:
            tempi = []
            for z in x:
                if(z in seenTeams):
                    tempi.append(bin(2**seenTeamsDict[z])[2:].zfill(len(seenTeamsDict)))
                else:
                    tempi.append(z)
            temp.append(tempi)
        self.gameList = temp
        
    def removeQuotes(self, string):
        if (string.startswith('"') and string.endswith('"')) or (string.startswith("'") and string.endswith("'")):
            return string[1:-1]
        return string  
    
    def addGame(self, team1, score1, stats1, team2, score2, stats2):
        self.gameList.append([team1, score1, stats1, team2, score2, stats2])
        
    #give it two teams, the scores, and it will add it to the matrix
    def addScore(self, team1, team2, score1, score2):
        '''
        for a team in top array, the index in the array corrisponds to the matrix column there located in
        for a team in side array, the index in the array corrisponds to the matrix row there located in
        '''
        #team 1 score entry
        try:
            row = self.sideArray.index(team2)    

        except:
            self.sideArray.append(team2)
            row = self.sideArray.index(team2)    

        try:
            col = self.topArray.index(team1)
        except:
            self.topArray.append(team1)
            col = self.topArray.index(team1)
        temp = self.sc[row, col]
        counter = 0
        for e in temp:
            if (e == -1):
                temp[counter] = score1
                break
            counter += 1
        self.sc[row, col] = temp
        
        #team 2 score entry
        try:
            row = self.sideArray.index(team1)    
        except:
            self.sideArray.append(team1)
            row = self.sideArray.index(team1)    
            
        try:
            col = self.topArray.index(team2)
        except:
            self.topArray.append(team2)
            col = self.topArray.index(team2)
        temp = self.sc[row, col]
        counter = 0
        for e in temp:
            if (e == -1):
                temp[counter] = score2
                break
            counter += 1
        self.sc[row, col] = temp
    
    #returns the score(s) for match up
    def getScore(self, team1, team2, gameSelect = None):
        print(team1, team2)
        try:
            score1 = self.sc[self.sideArray.index(team2), self.topArray.index(team1)]
            score2 = self.sc[self.sideArray.index(team1), self.topArray.index(team2)]
            if (gameSelect == None):
                print(team1, score1)
                print(team2, score2)
            else:
                print(team1, score1[gameSelect])
                print(team2, score2[gameSelect])
        except:
            print('Invalid input of teams')
    
    def getGameList(self):
        return copy.deepcopy(self.gameList)
    
    #constructs a matrix of the avg score in a matchup
    def buildAvgMatrix(self): 
        for col in range(len(self.sc[:,0])):   #depth
            for row in range(len(self.sc[0, :])):  #width
                tempScore = self.sc[row, col]
                avgScore = 0.0
                count = 0.0
                for j in tempScore:
                    if (j != -1):
                        avgScore += j
                        count += 1
                    else:
                        break
                try:
                    avgScore = avgScore / count
                except:
                    avgScore = -1
                self.am[row, col] = avgScore
    
    #get the value of the avg score for a match up
    def getAvgScore(self, team1, team2):
        try:
            score1 = self.am[self.sideArray.index(team2), self.topArray.index(team1)]
            score2 = self.am[self.sideArray.index(team1), self.topArray.index(team2)]
            print(team1, score1)
            print(team2, score2)        
        except:
            print('Invalid input of teams')

Extract the data form the file.

In [3]:
gameStats = GameStats(6, 10, [8, 50, 53, 54, 58], 3, 9, [5, 22, 25, 26, 30])
gameList = gameStats.getGameList() # get the list of games
#getting rid of the strings
def removeQuotes(gameList):
    for row in gameList:
        for x in range(len(row)):
            #convert scores strings to float
            if (x == 1 or x == 4): row[x] = float(row[x])
            #convert arrays to floats
            if (x == 2 or x == 5): row[x] = list(map(float, row[x]))
    return gameList


Print off all the data used into file name 'gameList.txt' so the user can see what stats were working with.

In [4]:
gameListFile = open("gameList.txt", "w")
for row in gameList:
    gameListFile.write('\n-------------------------------------------')
    gameListFile.write('\nHome: ')
    gameListFile.write(row[0])
    gameListFile.write('\nScore: ')
    gameListFile.write(str(row[1]))
    
    gameListFile.write('\n  game: ')
    gameListFile.write(str(row[2][0]))
    gameListFile.write('\n  hits: ')
    gameListFile.write(str(row[2][1]))
    gameListFile.write('\n  home runs: ')
    gameListFile.write(str(row[2][2]))
    gameListFile.write('\n  RBI: ')
    gameListFile.write(str(row[2][3]))
    gameListFile.write('\n  walks: ')
    gameListFile.write(str(row[2][4]))
    
    gameListFile.write('\n-----------------')
    
    gameListFile.write('\nAway: ')
    gameListFile.write((row[3]))
    gameListFile.write('\nScore: ')
    gameListFile.write(str(row[4]))
    
    gameListFile.write('\n  game: ')
    gameListFile.write(str(row[5][0]))
    gameListFile.write('\n  hits: ')
    gameListFile.write(str(row[5][1]))
    gameListFile.write('\n  home runs: ')
    gameListFile.write(str(row[5][2]))
    gameListFile.write('\n  RBI: ')
    gameListFile.write(str(row[5][3]))
    gameListFile.write('\n  walks: ')
    gameListFile.write(str(row[5][4]))
    gameListFile.write('\n-------------------------------------------')
gameListFile.close()
gameList = removeQuotes(gameList) #now thats its printed to the file, lets remove the quotes from the #

Now lets make the average stats for each team.

In [5]:
avgStats = dict()
np.set_printoptions(precision=4)  #IF you wanna remove this i recommend restarting the kernal

#add up all the stats for each team
for row in gameList:
    #home
    if row[0] in avgStats:
        avgStats[row[0]][0] = 0   #set the proir games to 0,because there set to right amount next line
        avgStats[row[0]] = np.sum([avgStats[row[0]], row[2]], axis=0)  #total stats + indivudal game stats
    else:
        avgStats[row[0]] = row[2]
    
    #away
    if row[3] in avgStats:
        avgStats[row[3]][0] = 0
        avgStats[row[3]] = np.sum([avgStats[row[3]], row[5]], axis=0)
    else:
        avgStats[row[3]] = row[5]

#divide the sum, to make the average..
keyList = avgStats.keys()    
for key in keyList:
    for index in range(1, len(avgStats[key])):  #divide all stats by games played (162, 162)
        avgStats[key][index] = avgStats[key][index] / avgStats[key][0]  

In [6]:
# split the stats up into different parts (this takes a lot of time, but I think it will be worth it)

#gameList = gameStats.getGameList() # I want to test what happens if we just use the stats of the game.
gameList = removeQuotes(gameList)   #because i initialized gamestats above, must remove quote again
homeTeamName = np.empty((0, len(list(gameList[0][0])))) 
homeTeamScore = np.empty((0, 1))
homeTeamStats = np.empty((0, len(gameList[0][2])))

visitingTeamName = np.empty((0, len(list(gameList[0][3])))) 
visitingTeamScore = np.empty((0, 1))
visitingTeamStats = np.empty((0, len(gameList[0][5])))

for row in gameList:
    homeTeamName = np.vstack((homeTeamName, list(row[0])))
    homeTeamScore = np.vstack((homeTeamScore, row[1]))
    homeTeamStats = np.vstack((homeTeamStats, row[2]))
    visitingTeamName = np.vstack((visitingTeamName, list(row[3])))
    visitingTeamScore = np.vstack((visitingTeamScore, row[4]))
    visitingTeamStats = np.vstack((visitingTeamStats, row[5]))

In [7]:
# take the stats and zscore them, I do a number of transformations on the array because of how the zScore input contstraints 
zScoredStatsHome = np.empty((0, len(homeTeamStats)))
zScoredStatsVisitor = np.empty((0, len(visitingTeamStats)))
for x in range(len(homeTeamStats[0])):
    zScoredStatsHome = np.vstack((zScoredStatsHome, stats.zscore([i[x] for i in homeTeamStats])))
    zScoredStatsVisitor = np.vstack((zScoredStatsVisitor, stats.zscore([i[x] for i in visitingTeamStats])))

homeTeamStatsTemp = np.empty((len(zScoredStatsHome[0]), 0))
visitingTeamStatsTemp = np.empty((len(zScoredStatsVisitor[0]), 0))
for i in zScoredStatsHome:
    homeTeamStatsTemp = np.hstack((homeTeamStatsTemp, [[x] for x in i]))

for i in zScoredStatsVisitor:
    visitingTeamStatsTemp = np.hstack((visitingTeamStatsTemp, [[x] for x in i]))

In [19]:
predFile = open("prediction.txt", "w")
predFile.close()     #clear the file
learnArr = [1*(10**(x)) for x in [-2]]
regulArr = [1*(2**(x)) for x in xrange(-10)]
layerArr = [5, 8]
widthArr = [30]
paramErr = dict()
mastErrList = []

for learnRate in tqdm_notebook(learnArr):
    for numWidth in tqdm_notebook(widthArr):
        for numLayers in tqdm_notebook(layerArr):                                  
            tf.reset_default_graph()
    
            # the input layer needs to have the same dimensions as our input (in this case the teams)
            homeTeamNameInput = tflearn.input_data(shape=[None, len(homeTeamName[0])], name='nameInput1')
            homeTeamStatsInput = tflearn.input_data(shape=[None, len(homeTeamStatsTemp[0])], name='statsInput1')
            visitingTeamNameInput = tflearn.input_data(shape=[None, len(visitingTeamName[0])], name='nameInput2')
            visitingTeamStatsInput = tflearn.input_data(shape=[None, len(visitingTeamStatsTemp[0])], name='statsInput2')
        
            nameProcess1 = tflearn.fully_connected(homeTeamNameInput, 4)
            nameProcess2 = tflearn.fully_connected(visitingTeamNameInput, 4)
            net = tflearn.layers.merge_ops.merge([nameProcess1, homeTeamStatsInput, nameProcess2, visitingTeamStatsInput], 'concat', axis=1)
            # next we have the hidden layer it is the feature matrix the size is arbitrary.
            for _ in range(numLayers):
                net = tflearn.fully_connected(net, numWidth)
            
            net = tflearn.dropout(net, keep_prob=0.5)
            # The output layer
            net = tflearn.fully_connected(net, 2)
            net = tflearn.regression(net, name='target', learning_rate=learnRate)
        
            # take only the stats for each team and put them into an array
            NNOutput = [[i[1], i[4]] for i in gameList]
            NNOutput = np.array(NNOutput)
        
            # Define model
            model = tflearn.DNN(net)
            # Start training (apply gradient descent algorithm)
            model.fit({'nameInput1':homeTeamName, 'statsInput1':homeTeamStatsTemp, 'nameInput2':visitingTeamName, 'statsInput2':visitingTeamStatsTemp}, NNOutput, validation_set=0.1, n_epoch=20, show_metric=True)
            
            
            #MAKING THE PREDICTIONS
            predFile = open("prediction.txt", "a")
            errorArr = []
            for i in tqdm_notebook(range(len(homeTeamName))):
                predict = model.predict([[homeTeamName[i]], [homeTeamStatsTemp[i]], [visitingTeamName[i]], [visitingTeamStatsTemp[i]]])
                pred = predict[0]   #only need 1 dimension in the array
                pred[0] = round(pred[0], 2)  #rounding the predictions
                pred[1] = round(pred[1], 2)
                #using relitive error but cant divide by 0, so replace actual score of 0 with 0.1
                if (NNOutput[i][0] == 0): NNOutput[i][0] = 0.1
                if (NNOutput[i][1] == 0): NNOutput[i][1] = 0.1
                
                error = (abs(pred[0] - NNOutput[i][0]) / NNOutput[i][0]) + (abs(pred[1] - NNOutput[i][1]) / NNOutput[i][1])  #adding the 'relitive error'
                error = round(error, 4)  #round to 4 digits
                errorArr.append(error)   
                if (i < 15):   #only print 15 cause printing the prediction for 1000 games isnt needed
                    predFile.write('\n-----------')
                    predFile.write('\n     Home  |  Away')
                    predFile.write('\nPred:  ' + str(pred[0]) + ' | '+ str(pred[1]))
                    predFile.write('\nActu:  ' + str(NNOutput[i][0]) + ' | '+ str(NNOutput[i][1]))
                    predFile.write('\nError: ' + str(error))
                    predFile.write('\n-----------')
            
            
            totErr = round(sum(errorArr) / len(homeTeamName), 3)  #the average relative errror
            tempStr = ('Error: ' + str(totErr) + ', learnRate: ' + str(learnRate) + ', Width: ' + str(numWidth) + ', layers: ' + str(numLayers))
            mastErrList.append(tempStr)  #this is an list of all the relitive error averages and what combination caused them
            
            predFile.write('\n^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^'+ str(len(mastErrList)-1)+ '^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n')
            predFile.write(tempStr + '\n')
            predFile.write('\n_________________________________________________________________________________')
            predFile.write('\n_________________________________________________________________________________')
            predFile.write('\n_________________________________________________________________________________')
            predFile.close()   #close the file so that it saves the changes

Training Step: 699  | total loss: 6.14998 | time: 0.273s
| Adam | epoch: 020 | loss: 6.14998 - acc: 0.4187 -- iter: 2176/2185
Training Step: 700  | total loss: 6.14890 | time: 1.284s
| Adam | epoch: 020 | loss: 6.14890 - acc: 0.3987 | val_loss: 6.44243 - val_acc: 0.1111 -- iter: 2185/2185
--



Here is a list of all the combinations tested and the 'average relitive error' they yeild.

In [22]:
for i in mastErrList:
    print(i)

Error: 1.993, learnRate: 0.01, Width: 30, layers: 5
Error: 9750.962, learnRate: 0.01, Width: 30, layers: 8
